In [34]:
# Get emails from Gmail and parse the papers and save it as excel file
%load_ext autoreload
%autoreload 2
from connect_to_service import *
from parse_gmail_message import *
from semantic_scholar import query_tldr_all_papers

import pandas as pd
import base64
import os.path as ospath

# Override the default parameters
DATA_FOLDER = "./data/"
PAPERS_LABEL = 'Subscribe/Gscholar'
SHEET_ID = '1Z5Riim21O7Ti5hHlWzriBhn2CbJibqee4psHkonCcBw'

# Create data folder if not exists
if not ospath.exists(DATA_FOLDER):
  ospath.makedirs(DATA_FOLDER)

# Call the Gmail API
creds = get_creds(DATA_FOLDER)
service = get_gmail_service(creds)

# Get all the messages with labels
labels = GetLabelsId(service,'me',[PAPERS_LABEL])
messages = ListMessagesWithLabels(service,"me",labels, maxResults=100)
print (f'Found {len(messages)} messages')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found 100 messages


In [58]:
# Parse the mails
from parse_gmail_message import PapersHTMLParser, PaperAggregator
pa = PaperAggregator()

for msg in messages:
  msg_content = GetMessage(service, "me", msg['id'])
  try:
    msg_str = base64.urlsafe_b64decode(msg_content['payload']['body']['data']).decode('utf-8', 'ignore').encode('ascii', 'ignore')
  except KeyError:
    continue

  msg_title = ''
  for h in msg_content['payload']['headers']:
    if h['name'] == 'Subject':
      msg_title = h['value']
  parser = PapersHTMLParser(msg_title)
  parser.parse(msg_str)
  parser.check_valid_papers()

  #for paper in parser.papers:
    #pa.add(paper)

In [24]:
# Change index to uuid
import uuid
from parse_gmail_message import PaperAggregator
import os.path as ospath

ns = uuid.uuid5(uuid.NAMESPACE_URL, 'gscholar')
pa = PaperAggregator()
pa.load_excel(ospath.join("data", 'archive.xlsx'))
df = pa.to_dataframe()